<a href="https://colab.research.google.com/github/shinobu357/TugasMLRaisya/blob/main/Week%2012/Week_12_Raisya_Athaya_Kamilah_101032380253_CIFAR_10_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from tqdm import tqdm

In [ ]:
# Data Preparation
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

100%|██████████| 170M/170M [00:04<00:00, 35.2MB/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [ ]:
# CNN Architecture
class CNN(nn.Module):
    def __init__(self, kernel_size=3, pooling_type='max'):
        super(CNN, self).__init__()
        self.pooling_type = pooling_type

        self.conv1 = nn.Conv2d(3, 32, kernel_size=kernel_size, padding=kernel_size // 2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=kernel_size, padding=kernel_size // 2)
        self.fc1 = nn.Linear(64 * 8 * 8, 128)
        self.fc2 = nn.Linear(128, 10)

        if pooling_type == 'max':
            self.pool = nn.MaxPool2d(2, 2)
        elif pooling_type == 'avg':
            self.pool = nn.AvgPool2d(2, 2)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 64 * 8 * 8)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [ ]:
# Training Function
def train_model(model, optimizer, criterion, train_loader, val_loader, epochs, patience=5, scheduler=None):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    best_acc = 0
    patience_counter = 0
    train_loss, val_loss, val_acc = [], [], []

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0

        for inputs, labels in tqdm(train_loader, desc=f'Epoch {epoch+1}/{epochs}'):
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        train_loss.append(running_loss / len(train_loader))

        # Validation Phase
        model.eval()
        correct = 0
        total = 0
        val_running_loss = 0.0

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_running_loss += loss.item()
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        val_loss.append(val_running_loss / len(val_loader))
        acc = 100 * correct / total
        val_acc.append(acc)

        # Early Stopping
        if acc > best_acc:
            best_acc = acc
            patience_counter = 0
        else:
            patience_counter += 1

        if patience_counter >= patience:
            print("Early stopping triggered!")
            break

        # Scheduler Step
        if scheduler:
            scheduler.step()

        print(f'Epoch {epoch+1}: Train Loss={train_loss[-1]:.4f}, Val Loss={val_loss[-1]:.4f}, Val Acc={val_acc[-1]:.2f}%')

    return train_loss, val_loss, val_acc

In [ ]:
results_sgd = []

kernel_sizes = [3, 5, 7]
pooling_types = ['max', 'avg']
epochs_list = [5, 50, 100, 250]
criterion = nn.CrossEntropyLoss()

for kernel_size in kernel_sizes:
    for pooling_type in pooling_types:
        model = CNN(kernel_size=kernel_size, pooling_type=pooling_type)
        optimizer = optim.SGD(model.parameters(), lr=0.001)
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.5)

        for num_epochs in epochs_list:
            print(f"SGD: Kernel={kernel_size}, Pooling={pooling_type}, Epochs={num_epochs}")
            train_loss, val_loss, val_acc = train_model(
                model, optimizer, criterion, train_loader, test_loader, num_epochs, scheduler=scheduler)

            results_sgd.append({
                'Kernel Size': kernel_size,
                'Pooling': pooling_type,
                'Epochs': num_epochs,
                'Train Loss': train_loss[-1],
                'Validation Loss': val_loss[-1],
                'Validation Accuracy': val_acc[-1]
            })


SGD: Kernel=3, Pooling=max, Epochs=5


Epoch 1/5: 100%|██████████| 782/782 [00:17<00:00, 45.79it/s]


Epoch 1: Train Loss=2.2965, Val Loss=2.2862, Val Acc=17.89%


Epoch 2/5: 100%|██████████| 782/782 [00:15<00:00, 51.19it/s]


Epoch 2: Train Loss=2.2733, Val Loss=2.2552, Val Acc=24.90%


...

Epoch 39/50: 100%|██████████| 782/782 [00:16<00:00, 48.31it/s]


Epoch 39: Train Loss=1.4221, Val Loss=1.4327, Val Acc=49.18%


Epoch 40/50: 100%|██████████| 782/782 [00:15<00:00, 50.36it/s]


Epoch 40: Train Loss=1.4140, Val Loss=1.4226, Val Acc=49.77%


Epoch 41/50:  38%|███▊      | 295/782 [00:06<00:09, 50.91it/s]

In [ ]:
results_rmsprop = []

kernel_sizes = [3, 5, 7]
pooling_types = ['max', 'avg']
epochs_list = [5, 50, 100,250]
criterion = nn.CrossEntropyLoss()

for kernel_size in kernel_sizes:
    for pooling_type in pooling_types:
        model = CNN(kernel_size=kernel_size, pooling_type=pooling_type)
        optimizer = optim.RMSprop(model.parameters(), lr=0.001)
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.5)

        for num_epochs in epochs_list:
            print(f"RMSProp: Kernel={kernel_size}, Pooling={pooling_type}, Epochs={num_epochs}")
            train_loss, val_loss, val_acc = train_model(
                model, optimizer, criterion, train_loader, test_loader, num_epochs, scheduler=scheduler)

            results_rmsprop.append({
                'Kernel Size': kernel_size,
                'Pooling': pooling_type,
                'Epochs': num_epochs,
                'Train Loss': train_loss[-1],
                'Validation Loss': val_loss[-1],
                'Validation Accuracy': val_acc[-1]
            })


RMSProp: Kernel=3, Pooling=max, Epochs=5


Epoch 1/5: 100%|██████████| 782/782 [00:15<00:00, 50.11it/s]


Epoch 1: Train Loss=1.3720, Val Loss=1.2856, Val Acc=53.40%


Epoch 2/5: 100%|██████████| 782/782 [00:15<00:00, 50.98it/s]


Epoch 2: Train Loss=1.0120, Val Loss=1.1430, Val Acc=59.87%


...

Epoch 8: Train Loss=0.0678, Val Loss=3.3277, Val Acc=68.04%


Epoch 9/250: 100%|██████████| 782/782 [00:17<00:00, 45.45it/s]


Epoch 9: Train Loss=0.0662, Val Loss=3.3528, Val Acc=67.50%


Epoch 10/250: 100%|██████████| 782/782 [00:18<00:00, 43.42it/s]


Early stopping triggered!


In [ ]:
# Define the loss function
criterion = nn.CrossEntropyLoss()

# Training the model with different configurations
results_adam = []

kernel_sizes = [3, 5, 7]
pooling_types = ['max', 'avg']
epochs_list = [5, 50, 100, 250]

for kernel_size in kernel_sizes:
    for pooling_type in pooling_types:
        model = CNN(kernel_size=kernel_size, pooling_type=pooling_type)
        optimizer = optim.Adam(model.parameters(), lr=0.001)
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.5)

        for num_epochs in epochs_list:
            print(f"Adam: Kernel={kernel_size}, Pooling={pooling_type}, Epochs={num_epochs}")
            train_loss, val_loss, val_acc = train_model(
                model, optimizer, criterion, train_loader, test_loader, num_epochs, scheduler=scheduler)

            results_adam.append({
                'Kernel Size': kernel_size,
                'Pooling': pooling_type,
                'Epochs': num_epochs,
                'Train Loss': train_loss[-1],
                'Validation Loss': val_loss[-1],
                'Validation Accuracy': val_acc[-1]
            })


Adam: Kernel=3, Pooling=max, Epochs=5


Epoch 1/5: 100%|██████████| 782/782 [00:17<00:00, 45.52it/s]


Epoch 1: Train Loss=1.3527, Val Loss=1.0623, Val Acc=62.39%


Epoch 2/5: 100%|██████████| 782/782 [00:15<00:00, 49.83it/s]


Epoch 2: Train Loss=0.9521, Val Loss=0.9190, Val Acc=66.99%


...

Epoch 10: Train Loss=0.0674, Val Loss=2.9735, Val Acc=67.25%


Epoch 11/250: 100%|██████████| 782/782 [00:17<00:00, 44.21it/s]


Epoch 11: Train Loss=0.0673, Val Loss=2.8807, Val Acc=68.07%


Epoch 12/250: 100%|██████████| 782/782 [00:17<00:00, 44.27it/s]


Early stopping triggered!
